In [2]:
import os 
import sys

sys.path.append(os.path.abspath('../..'))


In [3]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import DQN
from environment.env import Grid
from utils.checkpoints import find_last_checkpoint

obstacles = [((0.14625, 0.3325), (0.565, 0.55625)), 
             ((0.52875, 0.5375), (0.7375, 0.84125)), 
             ((0.0, 0.00125), (0.01625, 0.99125)), 
             ((0.0075, 0.00125), (0.99875, 0.04)), 
             ((0.98875, 0.0075), (0.99875, 1.0)), 
             ((0.00125, 0.9825), (0.99875, 1.0))]

env = Grid(
    obstacles=obstacles, 
    shear_range=(-.2, .2),
    stretch_range=(.4,1),
    render_mode="human"
)

last_checkpoint = find_last_checkpoint("../../agents/pretrained/MDP/DQN_continous_1vbmjd2a")
model = DQN.load(f"../../agents/pretrained/MDP/DQN_continous_1vbmjd2a/{last_checkpoint}", env=env)


mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=20, deterministic=True)
print(mean_reward, std_reward)
env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/flaccagora/.miniconda3/envs/robogym/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


-119.95 118.70361199222202
